In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def unique_words(s):
    unique = set(s.split(' ')) 
    return len(unique)

# def sentences(s):
#     sent = s.replace('|||',' ').split()
#     count = 0
#     for word in sent:
#         if len(word)>1:
#             if word[-1]=='.|!|?' :
#                 count+=1
#     return count

def emojis(post):
    # does not include emojis made purely from symbols, only :word:
    emoji_count = 0
    words = post.split()
    for e in words:
        if 'http' not in e:
            if e.count(':')==2:
                emoji_count+=1
    return emoji_count

def colons(post):
    # Includes colons used in emojis
    colon_count = 0
    words = post.split()
    for e in words:
        if 'http' not in e:
            colon_count+=e.count(':')
    return colon_count

def domains(post):
    # Includes colons used in emojis
    domains = []
    words = post.split(' ')
    for s in words:
#         print(s)
        if 'http' in s:
            s1 = s.split('.com')[0].split('//')[-1]
            s2 = s1[s1.rfind('.')+1:]
            if s2 not in domains:
                domains.append(s2)
    return domains

def grouper(df,group):
    return df.groupby(group).agg({
        'type':'count',
        'unique_words':['sum','mean','median'],
        'link_count':['sum','mean','median'],
        'word_count':['sum','mean','median'],
        'qm':['sum','mean','median'],
#         'sentences':['sum','mean','median']
    })

In [3]:
#Pull in csv
mbti_user = pd.read_csv('../data/mbti_1.csv')
#.reset_index().rename(columns={'index':'id'})

In [4]:
#Looking at words
mbti_user['unique_words'] = mbti_user['posts'].str.replace('|||', ' ').apply(unique_words)
mbti_user['word_count'] = mbti_user['posts'].apply(lambda s : s.count(' ')+1)

In [5]:
#looking at grammer
mbti_user['qm'] = mbti_user['posts'].apply(lambda s : s.count('?'))
mbti_user['em'] = mbti_user['posts'].apply(lambda s : s.count('!'))
mbti_user['colons'] = mbti_user['posts'].apply(colons)
mbti_user['emojis'] = mbti_user['posts'].apply(emojis)

In [11]:
#Looking at links
mbti_user['link_count'] = mbti_user['posts'].apply(lambda s : s.count('http'))
# mbti_user['unique_domains'] = mbti_user['posts'].replace('|||', ' ').apply(domains)
# mbti_user['unique_domains_count'] = mbti_user['unique_domains'].apply(len)
mbti_user['youtube_link'] = mbti_user['posts'].apply(lambda s : s.count('youtube') + s.count('youtu.be'))

In [7]:
#splitting out the different personality elements
mbti_user['extroverted'] = mbti_user['type'].apply(lambda s : 1 if s[0] == 'E' else 0)
mbti_user['sensor'] = mbti_user['type'].apply(lambda s : 1 if s[1] == 'S' else 0)
mbti_user['thinker'] = mbti_user['type'].apply(lambda s : 1 if s[2] == 'T' else 0)
mbti_user['judger'] = mbti_user['type'].apply(lambda s : 1 if s[3] == 'J' else 0)

In [12]:
mbti_user.head()

,type,posts,unique_words,word_count,qm,em,colons,emojis,link_count,youtube_link,extroverted,sensor,thinker,judger
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,77,578,18,3,7,2,24,16,0,0,0,1
1,ENTP,'I'm finding the lack of me in these posts ver...,77,1194,5,0,18,0,10,1,1,0,1,0
2,INTP,'Good one _____ https://www.youtube.com/wat...,72,869,12,4,13,0,5,3,0,0,1,0
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",74,1101,11,3,3,0,2,2,0,0,1,1
4,ENTJ,'You're fired.|||That's another silly misconce...,77,987,10,1,7,1,6,3,1,0,1,1


In [9]:
unique_domains = {}
for post in mbti_user['posts']:
    if 'http' in post:
        for s in post.split():
            if 'http' in s:
                s1 = s.split('.com')[0].split('//')[-1]
                s2 = s1[s1.rfind('.')+1:]
                if s2 not in unique_domains:
                    unique_domains[s2] = 1
                else:
                    unique_domains[s2] += 1

In [10]:
len(pd.DataFrame(unique_domains.items(), columns=['domain', 'count']).sort_values('count', ascending=False))

3218